In [58]:
def cos_sim(x,y):
    '''
    余弦相似性
    input:  x(mat):以行向量的形式存储，可以是用户或者商品
            y(mat):以行向量的形式存储，可以是用户或者商品
    output: x和y之间的余弦相似度
    '''
    numerator=np.dot(x,y.T)#x和y之间的内机
    denominator = np.sqrt(np.dot(x,x.T))*np.sqrt(np.dot(y,y.T))
    return (numerator/denominator)

In [71]:
def similarity(data):
    '''
    计算矩阵中任意两行之间的相似度
    input:  data(mat):任意矩阵
    output: w(mat):任意两行之间的相似度
    '''
    m = np.shape(data)[0]#用户的数量
    #初始化相似度矩阵
    w = np.mat(np.zeros((m,m)))
    for i in range(m):
        for j in range(i,m):
            if i!=j:
                #计算任意两行之间的相似度
                w[i,j] = cos_sim(data[i,],data[j,])
                w[j,i] = w[i,j]
            else:
                w[i,j]=0#约定自身的相似度为0
    return w


In [47]:
import numpy as np

In [68]:
A=[
    [4,3,0,5,0],
    [5,0,4,4,0],
    [4,0,5,0,3],
    [2,3,0,1,0],
    [0,4,2,0,5]
]

In [69]:
A=np.array(A)

In [72]:
similarity(A)

matrix([[0.        , 0.74926865, 0.32      , 0.83152184, 0.25298221],
        [0.74926865, 0.        , 0.74926865, 0.49559463, 0.1579597 ],
        [0.32      , 0.74926865, 0.        , 0.30237158, 0.52704628],
        [0.83152184, 0.49559463, 0.30237158, 0.        , 0.47809144],
        [0.25298221, 0.1579597 , 0.52704628, 0.47809144, 0.        ]])

##### 基于用户的协同过滤

In [112]:
def user_base_recommend(data,w,user):
    '''
     基于用户相似性为用户user推荐商品
     input:  data(mat):用户商品矩阵
            w(mat):用户之间的相似度
            user(int):用户的编号
     output: predict(list):推荐列表
    '''
    m,n=np.shape(data)
    interaction = data[user,]#用户user与商品信息
    #先找到用户user没有打过分的商品
    not_inter = []
    for i in range(n):#n商品数量
        if interaction[i]==0:#没有打分的商品
            not_inter.append(i)
    #print(not_inter)
    #对没有打分的商品进行预测
    predict={}
    for x in not_inter:
        item = np.copy(data[:,x])#找到所有用户对商品x的打分信息
        for i in range(m):#对所有的用户
            if item[i] != 0:#如果用户对商品x打过分
                if x not in predict:
                    predict[x]=w[user,i]*item[i]
                    
                else:
                    predict[x] = predict[x]+w[user,i]*item[i]
                    
        #按照预测的大小排序
    return sorted(predict.items(),key=lambda d:d[1],reverse=True)
            

In [113]:
user_base_recommend(A,w,0)

[(2, 5.10303902268836), (4, 2.2249110640673515)]

In [114]:
def top_k(predict,k):
    '''为用户推荐前k个商品
    input:  predict(list):排好序的商品列表
            k(int):推荐的商品个数
    output: top_recom(list):top_k个商品
    '''
    top_recom=[]
    len_r = len(predict)#计算有多少待推荐商品
    if k>=len_r:#如果需要推荐的个数大于待推荐的数量，直接输出
        top_recom=predict
    else:
        for i in range(k):
            top_recom.append(predict[i])
            
    return top_recom

In [121]:
k=2

In [122]:
result=top_k(user_base_recommend(A,w,0),k)

In [125]:
k=2
result=top_k(user_base_recommend(A,w,0),k)
for i in range(k):
    print("给User_0推荐的商品有:商品%d,打分为%.3f"%(result[i][0],result[i][1]))

给User_0推荐的商品有:商品2,打分为5.103
给User_0推荐的商品有:商品4,打分为2.225


##### 基于Item协同过滤

In [126]:
B=A.T

In [128]:
B

array([[4, 5, 4, 2, 0],
       [3, 0, 0, 3, 4],
       [0, 4, 5, 0, 2],
       [5, 4, 0, 1, 0],
       [0, 0, 3, 0, 5]])

In [129]:
similarity(B)

matrix([[0.        , 0.39524659, 0.76346445, 0.82977382, 0.26349773],
        [0.39524659, 0.        , 0.204524  , 0.47633051, 0.58823529],
        [0.76346445, 0.204524  , 0.        , 0.36803496, 0.63913749],
        [0.82977382, 0.47633051, 0.36803496, 0.        , 0.        ],
        [0.26349773, 0.58823529, 0.63913749, 0.        , 0.        ]])

In [131]:
def Item_base_recommend(data,w,user):
    '''基于商品相似度为用户user推荐商品
    input:  data(mat):商品用户矩阵
            w(mat):商品与商品之间的相似性
            user(int):用户的编号
    output: predict(list):推荐列表
    '''
    m,n=np.shape(data)#m是商品数量，n是用户数量
    interaction = data[:,user].T#用户user与商品信息
    #先找到用户user没有打过分的商品
    not_inter = []
    for i in range(n):#n商品数量
        if interaction[i]==0:#没有打分的商品
            not_inter.append(i)
    #print(not_inter)
    #对没有打分的商品进行预测
    predict={}
    for x in not_inter:
        item = np.copy(interaction)#找到用户对所有商品的打分信息
        for j in range(m):#对所有的用户
            if item[j] != 0:#如果用户对商品x打过分
                if x not in predict:
                    predict[x]=w[x,j]*item[j]
                    
                else:
                    predict[x] = predict[x]+w[x,j]*item[j]
                    
        #按照预测的大小排序
    return sorted(predict.items(),key=lambda d:d[1],reverse=True)
            

In [132]:
Item_base_recommend(B,similarity(B),0)

[(2, 5.507604598998138), (4, 2.8186967825714824)]